In [1]:
import os
from pathlib import Path
import joblib

import json
import requests
from numpy.random import randint

PROJECT_DIR = Path(os.getcwd()).resolve().parents[0]


# Load submission data for an example of an input
with open(PROJECT_DIR.joinpath('data/processed/dataset_processed/X_submission.joblib'), 'rb') as fin:
    X_submission = joblib.load(fin)
X_submission.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,www.facebook.com,2014-04-10 11:19:53,s-static.ak.facebook.com,2014-04-10 11:19:53,apis.google.com,2014-04-10 11:19:54,c1.adform.net,2014-04-10 11:19:54,www.google.com,2014-04-10 11:19:54,mpp2.vindicosuite.com,2014-04-10 11:19:54,api.nanigans.com,2014-04-10 11:19:54,www.google.fr,2014-04-10 11:19:54,secure.vistaprint.fr,2014-04-10 11:20:00,www.vistaprint.fr,2014-04-10 11:20:00
2,annotathon.org,2014-03-07 11:00:28,annotathon.org,2014-03-07 11:00:53,annotathon.org,2014-03-07 11:00:58,annotathon.org,2014-03-07 11:01:06,annotathon.org,2014-03-07 11:01:09,annotathon.org,2014-03-07 11:01:10,annotathon.org,2014-03-07 11:01:23,annotathon.org,2014-03-07 11:01:29,annotathon.org,2014-03-07 11:01:30,annotathon.org,2014-03-07 11:01:53
3,safebrowsing-cache.google.com,2014-05-12 15:55:12,safebrowsing-cache.google.com,2014-05-12 15:55:13,safebrowsing-cache.google.com,2014-05-12 15:55:14,safebrowsing-cache.google.com,2014-05-12 15:56:15,safebrowsing-cache.google.com,2014-05-12 15:56:16,safebrowsing-cache.google.com,2014-05-12 15:56:17,safebrowsing-cache.google.com,2014-05-12 15:56:18,safebrowsing-cache.google.com,2014-05-12 15:56:19,o2i.univ-bpclermont.fr,2014-05-12 15:56:33,o2i.univ-bpclermont.fr,2014-05-12 15:56:36
4,fr.yahoo.com,2014-04-11 10:03:19,yahoo.fr,2014-04-11 10:03:19,ocsp.verisign.com,2014-04-11 10:03:20,ocsp.thawte.com,2014-04-11 10:03:21,rapidssl-ocsp.geotrust.com,2014-04-11 10:03:21,clicks.beap.bc.yahoo.com,2014-04-11 10:03:22,ocsp.verisign.com,2014-04-11 10:03:22,ocsp.digicert.com,2014-04-11 10:03:22,ocsp.digicert.com,2014-04-11 10:03:23,clicks.beap.bc.yahoo.com,2014-04-11 10:03:23
5,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:31,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:32,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:33,go.microsoft.com,2014-05-16 15:05:39,windows.microsoft.com,2014-05-16 15:05:40,res2.windows.microsoft.com,2014-05-16 15:05:40,res1.windows.microsoft.com,2014-05-16 15:05:40,ajax.microsoft.com,2014-05-16 15:05:40,js.microsoft.com,2014-05-16 15:05:40,www.bing.com,2014-05-16 15:05:44


In [2]:
def sample_to_json_input(sample):
    values = {k: str(v) for k, v in sample.items()}
    
    result = {'session': []}
    for i in range(1, 10+1):
        site = values[f'site{i}']
        time = values[f'time{i}']
        result['session'].append({'site': site, 'time': time})
        
    return result

In [3]:
# Most samples belong to class 0. This is one of the samples with high probability of class 1
# Predicted probability should be 0.6793372790741214, as calculated earlier by make-submission step
sample_10614 = sample_to_json_input(X_submission.iloc[10614])
sample_10614

{'session': [{'site': 'ie-games.com', 'time': '2014-04-07 17:21:18'},
  {'site': 'ie-games.com', 'time': '2014-04-07 17:21:19'},
  {'site': 'um.simpli.fi', 'time': '2014-04-07 17:21:22'},
  {'site': 'soundwave.bnmla.com', 'time': '2014-04-07 17:21:22'},
  {'site': 'rp.gwallet.com', 'time': '2014-04-07 17:21:22'},
  {'site': 'www.facebook.com', 'time': '2014-04-07 17:21:25'},
  {'site': 'www.audienceinsights.net', 'time': '2014-04-07 17:21:40'},
  {'site': 'vk.com', 'time': '2014-04-07 17:21:41'},
  {'site': 'www.facebook.com', 'time': '2014-04-07 17:21:41'},
  {'site': 'youwatch.org', 'time': '2014-04-07 17:21:41'}]}

In [4]:
%%time
r = requests.post('http://192.168.99.100:5000/predict/', json=sample_10614)
print(json.dumps(r.json(), indent=4))

{
    "prediction": 1,
    "probability_class_1": 0.6793372790741214
}
Wall time: 22 ms


In [5]:
# Random sample
random_index = randint(X_submission.shape[0])
print(f'Sample {random_index}')
sample_random = sample_to_json_input(X_submission.iloc[random_index])
sample_random

Sample 64972


{'session': [{'site': 'ent.univ-bpclermont.fr', 'time': '2014-05-12 15:58:48'},
  {'site': 'www.univ-bpclermont.fr', 'time': '2014-05-12 15:58:53'},
  {'site': 'ent.univ-bpclermont.fr', 'time': '2014-05-12 15:59:31'},
  {'site': 'mail.google.com', 'time': '2014-05-12 15:59:47'},
  {'site': 'clients1.google.com', 'time': '2014-05-12 15:59:47'},
  {'site': 'clients1.google.com', 'time': '2014-05-12 15:59:51'},
  {'site': 'clients1.google.com', 'time': '2014-05-12 16:00:07'},
  {'site': 'www.parapentiste.info', 'time': '2014-05-12 16:00:08'},
  {'site': 'www.parapentiste.info', 'time': '2014-05-12 16:00:09'},
  {'site': 'connect.facebook.net', 'time': '2014-05-12 16:00:09'}]}

In [6]:
%%time
r = requests.post('http://192.168.99.100:5000/predict/', json=sample_random)
print(json.dumps(r.json(), indent=4))

{
    "prediction": 0,
    "probability_class_1": 0.0009161227797272831
}
Wall time: 21.5 ms
